In [63]:
import json
import pandas as pd
from pyvis.network import Network

In [93]:
names = pd.read_csv('../data/compound_info/compound', sep='\t', header=None, names=['KEGG', 'names'])
names['name'] = [sorted(ns, key=len)[0] for ns in names['names'].str.split(';')]
names.set_index('KEGG', inplace=True)

In [94]:
def build_graph_from_tree(tree_data):
    """
    Build a graph from a tree structure where each node corresponds to a unique hash,
    and edges are based on the connections in the tree.
    :param tree_data: The tree data (dict)
    :return: A PyVis network instance
    """
    net = Network(height='1200px', width='100%', directed=True)
    added_edges = set() 

    def traverse_and_add(node, parent_hash=None):
        """
        Recursively traverse the tree and add nodes and edges to the graph.
        :param node: Current node in the tree (dict)
        :param parent_hash: Hash of the parent node (if any)
        """
        current_hash = node['hash']
        label = ', '.join(names.loc[kegg_id, 'name'] for kegg_id in node['novel'])

        net.add_node(current_hash, label=label, color='red' if parent_hash is None else 'blue')

        if parent_hash is not None:
            edge = (parent_hash, current_hash, node.get('incoming', ''))
            if edge not in added_edges:
                net.add_edge(parent_hash, current_hash, label=edge[2])
                added_edges.add(edge)
                
        children = node.get('children', [])
        if children is None:
            return
        for child in children:
            traverse_and_add(child, current_hash)

    traverse_and_add(tree_data)
    return net


# Visualize the tree

In [100]:
with open('../data/searchspace.json') as f:
    d = json.load(f)
net = build_graph_from_tree(d)
net

<class 'pyvis.network.Network'> |N|=229 |E|=257

In [101]:
net.show('searchspace.html', notebook=False)

searchspace.html
